In [484]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import sys
import logging

To allow for flexible changes made some styling variables

In [485]:
logging.basicConfig(level=logging.INFO)
BACKGROUND_COLOR = '#242424'
FONT_COLOR = '#FFFFFF'
FONT_FAMILY = 'Lato'
PADDING = '10px'

H1_STYLE = {'font-size':'30px','text-align':'center'}
H3_STYLE = {'font-size':'14px'}

KPI_BOX_STYLE = {
    'textAlign': 'center',
    'padding': PADDING,
    'color': FONT_COLOR,
    'fontFamily': FONT_FAMILY,
    'backgroundColor': '#1E88E5',
    'borderRadius': '10px',
    'height': '100%',
    'display': 'flex',
    'justifyContent': 'center',
    'alignItems': 'center'
}

In [486]:
def generate_data(num_rows=1000):
    np.random.seed(0)
    years = np.random.choice(range(2000, 2024), num_rows)
    countries = np.random.choice(['Germany', 'France', 'Italy', 'Spain', 'UK', 'Belgium', 'Netherlands', 'Denmark', 'Sweden', 'Norway', 'Finland'], num_rows)
    construction_values = np.random.uniform(1000, 50000, num_rows)
    labor_wages = np.random.uniform(10, 50, num_rows)
    building_costs = np.random.uniform(500, 30000, num_rows)

    return pd.DataFrame({'Year': years, 'Country': countries,
                         'Construction Value': construction_values,
                         'Labor Wages': labor_wages,
                         'Building Cost': building_costs})

In [487]:
df = generate_data()

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

In [488]:
def create_kpi_box(box_id):
    return dbc.Col(html.Div(id=box_id, style=KPI_BOX_STYLE), width=3)

In [489]:
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value')]
)
def update_scatter_plot(year, countries):
    try:
        df_filtered = df[(df['Year'] == year) & (df['Country'].isin(countries))]

        scatter_plot = go.Scatter(
            x=df_filtered['Country'],
            y=df_filtered['Construction Value'],
            mode='markers'
        )

        layout = go.Layout(
            title='Construction Value by Country',
            xaxis=dict(title='Country'),
            yaxis=dict(title='Construction Value'),
            autosize=True
        )

        return {'data': [scatter_plot], 'layout': layout}
    except Exception as e:
        logging.exception(e)

In [490]:
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('year-slider', 'value'),
     Input('top-10-check', 'value'),
     Input('country-dropdown', 'value')]
)
def update_bar_chart(year, check_values, countries):
    try:
        df_filtered = df[(df['Year'] == year) & (df['Country'].isin(countries))]
        if 'top-10' in check_values:
            df_filtered = df_filtered.sort_values('Construction Value', ascending=False)[:10]

        bar_chart = go.Bar(
            x=df_filtered['Country'],
            y=df_filtered['Construction Value'],
        )

        layout = go.Layout(
            title='Construction Value by Country',
            xaxis=dict(title='Country'),
            yaxis=dict(title='Construction Value'),
            autosize=True
        )

        return {'data': [bar_chart], 'layout': layout}
    except Exception as e:
        logging.exception(e)

In [491]:
@app.callback(
    Output('histogram', 'figure'),
    [Input('year-slider', 'value'),
     Input('wage-cost-radio', 'value'),
     Input('country-dropdown', 'value')]
)
def update_histogram(year, radio_value, countries):
    try:
        df_filtered = df[(df['Year'] == year) & df['Country'].isin(countries)]
        histogram = go.Histogram(
            x=df_filtered[radio_value],
        )

        layout = go.Layout(
            title=f'{radio_value} Distribution',
            xaxis=dict(title=radio_value),
            yaxis=dict(title='Frequency'),
            autosize=True
        )

        return {'data': [histogram], 'layout': layout}
    except Exception as e:
        logging.exception(e)

In [492]:
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value')]
)
def update_pie_chart(year, countries):
    try:
        df_filtered = df[(df['Year'] == year) & (df['Country'].isin(countries))]

        pie_chart = go.Pie(
            labels=df_filtered['Country'],
            values=df_filtered['Construction Value'],
            hole=.3
        )

        layout = go.Layout(
            title='Construction Value by Country',
            autosize=True
        )

        return {'data': [pie_chart], 'layout': layout}
    except Exception as e:
        logging.exception(e)

In [493]:
@app.callback(
    [Output('kpi-1', 'children'),
     Output('kpi-2', 'children'),
     Output('kpi-3', 'children'),
     Output('kpi-4', 'children')],
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value')]
)
def update_kpi_boxes(year, countries):
    df_filtered = df[(df['Year'] == year) & df['Country'].isin(countries)]

    total_construction_value = round(df_filtered['Construction Value'].sum(),2)
    avg_labor_wages = round(df_filtered['Labor Wages'].mean(),2)
    avg_building_cost = round(df_filtered['Building Cost'].mean(),2)
    n_countries = len(df_filtered['Country'].unique())

    return f'Total Construction Value: €{total_construction_value}', f'Average Labor Wages: €{avg_labor_wages}', f'Average Building Cost: €{avg_building_cost}', f'Number Countries: {n_countries}'


In [494]:
app.layout = html.Div(children=[
    dbc.Container(
        [
            dbc.Row(
                [
                    dbc.Col(
                        [
                            html.H1("Construction Analytics Dashboard", style=H1_STYLE),
                            html.Hr(),
                            dbc.Row(
                                [
                                    create_kpi_box('kpi-1'),
                                    create_kpi_box('kpi-2'),
                                    create_kpi_box('kpi-3'),
                                    create_kpi_box('kpi-4'),
                                ]
                            ),
                            html.Hr(),
                            html.H3("Select Countries", style=H3_STYLE),
                            dcc.Dropdown(
                                id='country-dropdown',
                                options=[{'label': i, 'value': i} for i in df['Country'].unique()],
                                multi=True,
                                value=df['Country'].unique().tolist()  # Default value is now all countries
                            ),
                            html.Hr(),
                            html.H3("Select Year", style=H3_STYLE),
                            dcc.Slider(
                                id='year-slider',
                                min=df['Year'].min(),
                                max=df['Year'].max(),
                                value=df['Year'].min(),
                                marks={str(year): str(year) for year in df['Year'].unique()},
                                step=None
                            ),
                            html.Hr(),
                            dbc.Row(
                                [
                                    dbc.Col(
                                        [
                                            html.H3("Scatter Plot", style=H3_STYLE),
                                            html.Hr(),
                                            dcc.Graph(id='scatter-plot')
                                        ]
                                    )
                                ]
                            ),
                            html.Hr(),
                            dbc.Row(
                                [
                                    dbc.Col(
                                        [
                                            html.H3("Bar Chart", style=H3_STYLE),
                                            html.Hr(),
                                            dcc.Checklist(
                                                id='top-10-check',
                                                options=[{'label': 'Top 10', 'value': 'top-10'}],
                                                value=[]
                                            ),
                                            dcc.Graph(id='bar-chart')
                                        ]
                                    )
                                ]
                            ),
                            html.Hr(),
                            dbc.Row(
                                [
                                    dbc.Col(
                                        [
                                            html.H3("Histogram", style=H3_STYLE),
                                            html.Hr(),
                                            dcc.RadioItems(
                                                id='wage-cost-radio',
                                                options=[{'label': i, 'value': i} for i in ['Labor Wages', 'Building Cost']],
                                                value='Labor Wages'
                                            ),
                                            dcc.Graph(id='histogram')
                                        ]
                                    )
                                ]
                            ),
                            html.Hr(),
                            dbc.Row(
                                [
                                    dbc.Col(
                                        [
                                            html.H3("Pie Chart", style=H3_STYLE),
                                            html.Hr(),
                                            dcc.Graph(id='pie-chart')
                                        ]
                                    )
                                ]
                            )
                        ]
                    )
                ]
            )
        ]
    )
])

Finally run the app, it will run on your local network, e.g. http://127.0.0.1:XXXX/

In [495]:
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [29/May/2023 14:04:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 14:04:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2023 14:04:05] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 14:04:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2023 14:04:05] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 14:04:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2023 14:04:05] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 14:04:05] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2023 14:04:05] "GET /_d